# Code interpreting with Fireworks
This example uses the E2B's [Code Interpreter](https://github.com/e2b-dev/code-interpreter) as a tool for [Firefunction-v2](https://fireworks.ai/models/fireworks/firefunction-v2) by Fireworks.

We use the [secure sandbox](https://e2b.dev/docs/sandbox/overview) given by E2B to execute code for accomplishing the task given to the Firefunction-v2.
>

### 1. Installations
First, we install Fireworks, OpenAI, and the E2B Code Interpreter SDK.

In [1]:
%pip install --upgrade fireworks-ai openai e2b_code_interpreter

  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
Using cached packaging-24.1-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 23.2
    Uninstalling packaging-23.2:
      Successfully uninstalled packaging-23.2
  Attempting uninstall: openai
    Found existing installation: openai 1.45.0
    Uninstalling openai-1.45.0:
      Successfully uninstalled openai-1.45.0
  Attempting uninstall: fireworks-ai
    Found existing installation: fireworks-ai 0.14.0
    Uninstalling fireworks-ai-0.14.0:
      Successfully uninstalled fireworks-ai-0.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.2.5 requires packaging<24.0,>=23.2, but you have packaging 24.1 which is incompatible.
langchain-community 0.0.19 requires langchain-core<0.2,>=0.1.21, but you have langchain-core 0.2.5 which is 

### 2. Imports and API keys

You need to get your API keys and save them in .env file. You can copy and rename the .env.template file. We import all necessary libraries.

This is what you should put in the .env file:

```sh
# TODO: Get your Fireworks API key from https://fireworks.ai/api-keys
FIREWORKS_API_KEY = "your_api_key"

# TODO: Get your E2B API key from https://e2b.dev/docs
E2B_API_KEY = "your_api_key"
```

In [2]:
import openai
import json
import sys
import os
from dotenv import load_dotenv
load_dotenv()

FIREWORKS_API_KEY = os.getenv("FIREWORKS_API_KEY")
E2B_API_KEY = os.getenv("E2B_API_KEY")

### 3. Prompts and tools setup

Now we specify a system prompt, assigning a role to LLM, and user prompt for the task we want to acomplish.

In the ```tools```, we describe the functions available to the agent in great detail in JSON Schema. Here, it's just the ```execute_python``` function, the one that will execute the code generated by the LLM assistant.

In [3]:
SYSTEM_PROMPT = """
## your job & context
you are a python data scientist. you are given tasks to complete and you run python code to solve them.
You DO NOT MAKE SYNTAX MISTAKES OR FORGET ANY IMPORTS
- the python code runs in jupyter notebook.
- every time you call `execute_python` tool, the python code is executed in a separate cell. it's okay to multiple calls to `execute_python`.
- display visualizations using matplotlib or any other visualization library directly in the notebook. don't worry about saving the visualizations to a file.
- you have access to the internet and can make api requests.
- you also have access to the filesystem and can read/write files.
- you can install any pip package (if it exists) if you need to but the usual packages for data analysis are already preinstalled.
- you can run any python code you want, everything is running in a secure sandbox environment.
"""

USER_PROMPT = "Plot a 3D chart of sin x cos y please."

client = openai.OpenAI(
    base_url = "https://api.fireworks.ai/inference/v1",
    api_key = FIREWORKS_API_KEY
)

tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_python",
            "description": "Execute python code in a Jupyter notebook cell and returns any result, stdout, stderr, display_data, and error.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The python code to execute in a single cell.",
                    },
                },
                "required": ["code"],
            },
        },
        },
]

### 4. Defining the function for running the AI-generated code

We define the `code_interpret` function that uses the [E2B Code Interpreter SDK](https://e2b.dev/docs/code-interpreter/installation) to run AI-generated Python code.

We will use `exec_cell` to execute the AI-generated code inside the code interpreter sandbox.

Learn more about the `exec_cell` method [here](https://e2b.dev/docs/code-interpreter/execution).

In [4]:
def code_interpret(e2b_code_interpreter, code):
  print("Running code interpreter...")
  exec = e2b_code_interpreter.notebook.exec_cell(
    code,
    # Stream stdout and stderr from the Code Interpreter
    on_stdout=lambda stdout: print("[Code Interpreter]", stdout),
    on_stderr=lambda stderr: print("[Code Interpreter]", stderr),
    # You can also stream code execution results
    # on_result=...
  )

  if exec.error:
    print("[Code Interpreter ERROR]", exec.error)
  else:
    return exec

### 5. Getting agent's response

Now we define a ```chat``` function that uses the Firefunction-v2 model to get agent's response to the user's task. If the response contains a tool call of `execute_python`, the code interpreter by E2B is used to securely run the code.

In [5]:
def chat(e2b_code_interpreter, user_message):
  print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")
  
  messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": user_message}
  ]

  response = client.chat.completions.create(
    model = "accounts/fireworks/models/firefunction-v2",
    messages=messages,
    tools=tools,
    tool_choice="auto"
  )
  for choice in response.choices:
    if choice.message.tool_calls and len(choice.message.tool_calls) > 0:
      for tool_call in choice.message.tool_calls:
        if tool_call.function.name == "execute_python":
          if "code" in tool_call.function.arguments:
            args = json.loads(tool_call.function.arguments)
            code = args["code"]          
            print("CODE TO RUN")
            print(code)
            execution_output = code_interpret(e2b_code_interpreter, code)
            return execution_output
    else:
      print("Answer:", choice.message.content)


### 6. Printing the output

Finally we get the task result, calling the ```chat``` function.

Read more about how the execution with Code Interpreter SDK looks like [here](https://e2b.dev/docs/code-interpreter/execution).

In [6]:
from e2b_code_interpreter import CodeInterpreter

with CodeInterpreter(api_key=E2B_API_KEY) as code_interpreter:
  execution_output = chat(
    code_interpreter,
    USER_PROMPT,
  )

  if execution_output == None:
    print("No output from the code interpreter. Did the LLM generate any code?")
    sys.exit(1)

  # Access stdout
  print("Stdout:", execution_output.logs.stdout)
  # Access stderr
  print("Stderr:", execution_output.logs.stderr)

  # Access any runtime errors
  print("AI-generated Python runtime error:", execution_output.error)

  # Access any results of code execution - charts, interpreter last line, images, etc.
  print("Results:", execution_output.results)
  if len(execution_output.results) == 0:
    sys.exit(0)

execution_output.results[0]

ImportError: cannot import name 'EnvVars' from 'e2b' (/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/e2b/__init__.py)